In [6]:
import pandas as pd
import sys
import py_entitymatching as em

In [32]:
import dask
import dask.dataframe as dd

In [4]:
sys.path.append('/Users/pradap/Documents/Research/Python-Package/scaling/dmagellan/')

In [5]:
from dmagellan.core.downsample import *

In [31]:
AP = pd.read_csv('A.csv')

In [37]:
AD = dd.read_csv('A.csv', blocksize=200)

In [54]:
def preprocess_table(dataframe, idcol):
    strcols = list(get_str_cols(dataframe))
    strcols.append(idcol)
    projdf = dataframe[strcols]
    objsc = StringContainer()
    for row in projdf.itertuples():
        colvalues = row[1:-1]
        uid = row[-1]
        strings = [colvalue.strip() for colvalue in colvalues if not pd.isnull(colvalue)]
        concat_row = ' '.join(strings)
        concat_row = concat_row.translate(None, string.punctuation)
        objsc.push_back(uid, str2bytes(concat_row))
    return objsc

def tokenize_strings(objsc, stopwords):
    n = objsc.size()
    objtc = TokenContainer()
    objtc.tokenize(objsc, stopwords)
    return objtc

def build_inv_index(objtc):
    inv_obj = InvertedIndex()
    inv_obj.build_inv_index(objtc)
    return inv_obj

def probe(objtc, ids, objinvindex, yparam):
    objprobe = Prober()
    objprobe.probe(objtc, ids, objinvindex, yparam)
    return objprobe

def postprocess(result_list, ltable, rtable):
    lids = set()
    rids = set()
    for i in range(len(result_list)):
        result = result_list[i]
        lids.update(result.get_lids())
        rids.update(result.get_rids())
    lids = sorted(lids)
    rids = sorted(rids)
    return (ltable.loc[lids], rtable.loc[rids])

In [55]:
lstrings_d = preprocess_table(AD, 'id')

In [56]:
lstrings_d.get(0)

(0, 'a1 Kevin Smith 607 From St San Francisco')